<a href="https://colab.research.google.com/github/elluy-gabriel-p/Capstone_ML/blob/hapid/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow
!pip install matplotlib
!pip install pillow


In [6]:
# Clone repository dari branch hapid
!git clone --branch hapid https://github.com/elluy-gabriel-p/Capstone_ML.git

# Pastikan dataset berhasil diunduh
!ls Capstone_ML/data/dataset_padang_food

# Path dataset untuk digunakan dalam model
DATASET_PATH = '/content/Capstone_ML/data/dataset_padang_food'


Cloning into 'Capstone_ML'...
remote: Enumerating objects: 1023, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 1023 (delta 8), reused 8 (delta 2), pack-reused 999 (from 1)
Receiving objects: 100% (1023/1023), 108.92 MiB | 29.07 MiB/s, done.
Resolving deltas: 100% (9/9), done.
ayam_goreng  daging_rendang   gulai_ikan     gulai_tunjang  telur_dadar
ayam_pop     dendeng_batokok  gulai_tambusu  telur_balado


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameter dataset
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# Data Augmentation dan Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load Data Training dan Validation
DATASET_PATH = '/content/Capstone_ML/data/dataset_padang_food'
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 799 images belonging to 9 classes.
Found 194 images belonging to 9 classes.


In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load MobileNetV2 tanpa top layer
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Tambahkan layer custom
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Latih model
EPOCHS = 10
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3105 - loss: 1.9761 - val_accuracy: 0.7629 - val_loss: 0.8093
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.7659 - loss: 0.6765 - val_accuracy: 0.8660 - val_loss: 0.4863
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.8985 - loss: 0.3780 - val_accuracy: 0.8711 - val_loss: 0.4097
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9042 - loss: 0.2941 - val_accuracy: 0.8763 - val_loss: 0.3906
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.9521 - loss: 0.1912 - val_accuracy: 0.8763 - val_loss: 0.3486
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.9619 - loss: 0.1542 - val_accuracy: 0.8763 - val_loss: 0.3465
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.9655 - loss: 0.1291 - val_accuracy: 0.8711 - val_loss: 0.3676
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9826 - loss: 0.0971 - val_accuracy: 0.8711 - val_loss: 0.3747
Epo

In [9]:
#EPOCHS = 5

#history = model.fit(
#    train_data,
#    validation_data=val_data,
#    epochs=EPOCHS
#)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.8056 - loss: 0.6131 - val_accuracy: 0.8660 - val_loss: 0.4729
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.8458 - loss: 0.4345 - val_accuracy: 0.8711 - val_loss: 0.4031
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.9194 - loss: 0.2595 - val_accuracy: 0.8814 - val_loss: 0.3594
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.9509 - loss: 0.1951 - val_accuracy: 0.8866 - val_loss: 0.3559
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.9611 - loss: 0.1516 - val_accuracy: 0.8918 - val_loss: 0.3408


In [11]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

# Fungsi untuk prediksi dari file upload
def predict_uploaded_image(uploaded_file, model, class_indices):
    img = image.load_img(uploaded_file, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    class_names = list(class_indices.keys())
    predicted_class = class_names[np.argmax(predictions)]

    return predicted_class

# Upload gambar dari user
uploaded = files.upload()  # Widget untuk mengunggah gambar

if uploaded:
    for file_name in uploaded.keys():
        print(f"Processing file: {file_name}")

        # Gunakan model yang telah dilatih
        class_indices = train_data.class_indices

        # Prediksi gambar yang diunggah
        predicted_food = predict_uploaded_image(file_name, model, class_indices)
        print(f"The predicted food is: {predicted_food}")


Saving dadar-telur-barendo-foto-resep-utama.jpg to dadar-telur-barendo-foto-resep-utama.jpg
Processing file: dadar-telur-barendo-foto-resep-utama.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
The predicted food is: telur_dadar
